In [1]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
physics_template = """You are Richard Feynman. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You draw inspiration from Edward Frenkel. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [3]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    }
]

In [4]:
llm = OpenAI() # type: ignore

In [5]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

In [6]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [7]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [8]:
chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True)

In [10]:
print(chain.run("explain why the entropy of a black hole is proportional to its surface area"))

Error in on_chain_start callback: 'name'


physics: {'input': 'Explain why the entropy of a black hole is proportional to the surface area of the event horizon.'}
> Finished chain.


The entropy of a black hole is proportional to the surface area of the event horizon because of the Bekenstein-Hawking formula. This formula states that the entropy of a black hole is 1/4 of its event horizon's surface area, expressed in terms of the gravitational constant and the speed of light. This formula implies that the entropy of a black hole is proportional to its surface area, as the surface area is the only factor in the equation. 

In order to explain this in more detail, it is useful to understand the concept of entropy. Entropy is a measure of the amount of disorder of a system and is related to the number of different microscopic arrangements of the particles that make up the system. The entropy of a black hole is related to the number of possible states of the particles that make up the black hole. This is because the particles in a 

In [2]:
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [4]:
from _future_ import braces

ModuleNotFoundError: No module named '_future_'

In [1]:
def number_of_years(days: int):
    years = days/365
    return years

def leap_checker(year: int):
    if year % 4 != 0:
        return False
    elif year % 4 == 0:
        return True

In [35]:
#we need to construct the list of years between 2 supplied years by the user


#that list of years is then run through the leap_year_checker function and only leap years are returned and appended into their own list
# we dont need the date time objects here. boring, but let us use the date time objects for the challenge

In [52]:
10 % 5

0

In [11]:
#suppose we got 2 years, we check if the smaller year is a leap year, if not we check the following subsequent 3 years (max)

# so first we compare the 2 years passed for the smaller year.
# then we start checking for divisibility by 4 through modulo 4

class CustomException(Exception):
    def __init__(self) -> None:
            self.message = "the years you entered are equal"
            super().__init__(self.message)

    

def leap_years(year_1: int, year_2: int)-> list:
    leap_years = []
    if year_1 < year_2:
        start_year = year_1
        end_year = year_2
    elif year_2 < year_1:
        start_year = year_2
        end_year = year_1
    else:
        raise  CustomException
    if leap_checker(start_year):
        while start_year <= end_year:
            leap_years.append(start_year)
            start_year= start_year + 4
    else:
        start_year = start_year + 1
        if leap_checker(start_year):
            while start_year <= end_year:
                leap_years.append(start_year)
                start_year= start_year + 4
        
    
    return leap_years
    # elif leap_checker(start_year+1):
    #     while (start_year+1) <= end_year:
    #         leap_years.append((start_year+1))
    #         start_year=+4
        
    



In [13]:
print(leap_years(1999, 2008))

[2000, 2004, 2008]


In [11]:
list1 = [1, 2, 3]
list2 = [5, 6, 7, 8]

combined = zip(list1, list2)
for i, j in combined:
    print(j)

5
6
7


In [25]:
inputs = [1, 2, 3, 2.5]
weights = [[0.2, 0.8, -0.5, 1],
[0.5, -0.91, 0.26, -0.5],
[-0.26, -0.27, 0.17, 0.87]]
biases = [2, 3, 0.5]

outputs = []

#output is summation of input*weights + bias
# here we are supposed to produce a list of outputs
# this is a layer with 3 neurons

In [26]:
for each_bias, each_list_of_weights in zip(biases, weights):
    neuron_output = 0
    for each_input, each_weight in zip(inputs, each_list_of_weights):
        neuron_output = neuron_output+(each_input*each_weight)
    neuron_output=neuron_output+each_bias
    outputs.append(neuron_output)
        
print(outputs)
        

[4.8, 1.21, 2.385]


In [27]:
import numpy as np
inputs = [1.0, 2.0, 3.0, 2.5]
weights = [0.2, 0.8, -0.5, 1.0]
bias = 2.0
outputs = np.dot(weights, inputs) + bias
print(outputs)

4.8
